In [4]:
# Install the 'ucimlrepo' package, which allows you to easily access datasets
# from the UCI Machine Learning Repository.
# (Run this only once per environment — remove the '!' if running in a script.)
!pip install ucimlrepo  

In [5]:
# Import the function 'fetch_ucirepo' from the ucimlrepo package.
# This function is used to download datasets by their ID number.
from ucimlrepo import fetch_ucirepo 

# Fetch the dataset with ID = 2, which corresponds to the "Adult" dataset
# (also called the "Census Income" dataset).
adult = fetch_ucirepo(id=2)

# Extract the data (features and target) from the downloaded dataset.
# 'features' are the input variables (e.g., age, education, hours per week, etc.)
# 'targets' are what we want to predict (e.g., whether income > $50K).
X = adult.data.features 
y = adult.data.targets 

# Print metadata (general information) about the dataset.
# This usually includes the dataset name, description, number of rows/columns, etc.
print(adult.metadata)

# Print details about each variable (feature) in the dataset.
# This helps you understand the data types and what each column represents.
print(adult.variables)


{'uci_id': 2, 'name': 'Adult', 'repository_url': 'https://archive.ics.uci.edu/dataset/2/adult', 'data_url': 'https://archive.ics.uci.edu/static/public/2/data.csv', 'abstract': 'Predict whether annual income of an individual exceeds $50K/yr based on census data. Also known as "Census Income" dataset. ', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 48842, 'num_features': 14, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'], 'target_col': ['income'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1996, 'last_updated': 'Tue Sep 24 2024', 'dataset_doi': '10.24432/C5XW20', 'creators': ['Barry Becker', 'Ronny Kohavi'], 'intro_paper': None, 'additional_info': {'summary': "Extraction was done by Barry Becker from the 1994 Census database.  A set of reasonably clean records was extracted using the fol